In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter, defaultdict
from functools import partial
from pathlib import Path

import awkward as ak
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import pod5
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/barcode.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Duplex pairing

In [ ]:
pod5_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.pod5"
gaf_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.gaf"

In [ ]:
%%time
# gaf = pa.Table.from_batches([next(sio.iter_gaf(gaf_filename))])
# gaf = pa.Table.from_batches(list(it.islice(sio.iter_gaf(gaf_filename), 2)))
gaf = pa.Table.from_batches(sio.iter_gaf(gaf_filename))

In [ ]:
%%time
gaf_simplex = gaf.filter(pc.invert(pc.match_substring(gaf["name"], ";")))

In [ ]:
name_col = gaf_simplex["name"]  # [:10]

In [ ]:
%%time
hex_hyphenated = ak.from_arrow(name_col)

In [ ]:
idxs = list(set(range(36)) - set([8, 13, 18, 23]))

In [ ]:
%%time
hex = hex_hyphenated[:, idxs]

In [ ]:
hex_to_nibble = np.zeros(256, dtype=np.uint8)
for char in range(ord("0"), ord("9") + 1):
    hex_to_nibble[char] = char - ord("0")
for char in range(ord("a"), ord("f") + 1):
    hex_to_nibble[char] = char - ord("a") + 10
for char in range(ord("A"), ord("F") + 1):
    hex_to_nibble[char] = char - ord("A") + 10


@numba.njit(nogil=True)
def parse_hex(s):
    num_bytes, remainder = divmod(len(s), 2)
    if remainder != 0:
        return None
    ary = np.empty(num_bytes, dtype=np.uint8)
    for num_byte in range(num_bytes):
        ary[num_byte] = (hex_to_nibble[ord(s[2 * num_byte])] << 4) + hex_to_nibble[
            ord(s[2 * num_byte + 1])
        ]
    return ary


parse_hex(hex[0])

In [ ]:
hex_to_nibble

In [ ]:
(ord("0"), ord("9"), ord("a"), ord("f"), ord("A"), ord("F"))

In [ ]:
np.frombuffer(bytes.fromhex(hex[0]), dtype=np.uint8)

In [ ]:
hex.to_numpy()

In [ ]:
hex_to_nibble = np.zeros(256, dtype=np.uint8)
for char in range(ord("0"), ord("9") + 1):
    hex_to_nibble[char] = char - ord("0")
for char in range(ord("a"), ord("f") + 1):
    hex_to_nibble[char] = char - ord("a") + 10
for char in range(ord("A"), ord("F") + 1):
    hex_to_nibble[char] = char - ord("A") + 10


@numba.njit(nogil=True)
def parse_hex_array(ary):
    num_bytes, remainder = divmod(len(ary[0]), 2)
    if remainder != 0:
        return None
    out = np.empty((len(ary), num_bytes), dtype=np.uint8)
    for idx in range(len(ary)):
        for num_byte in range(num_bytes):
            out[idx, num_byte] = (
                hex_to_nibble[ary[idx][2 * num_byte]] << 4
            ) + hex_to_nibble[ary[idx][2 * num_byte + 1]]
    return out


parse_hex_array(hex)

In [ ]:
%timeit parse_hex_array(hex)

In [ ]:
reader = pod5.Reader(pod5_filename)

In [ ]:
read_table = reader.read_table.read_all()

In [ ]:
read_table.column_names

In [ ]:
read_table2 = read_table.select(["read_id", "start", "well", "channel"])

In [ ]:
read_table_df = pl.from_arrow(read_table2).lazy()

In [ ]:
gaf_df = pl.from_arrow(gaf).lazy()

In [ ]:
gaf_df.columns

In [ ]:
gaf["name"]

In [ ]:
n = ak.from_arrow(gaf["name"])

In [ ]:
n

In [ ]:
joined_df = read_table_df.join(gaf_df, left_on="read_id", right_on="name", how="left")

In [ ]:
joined_df.collect()

In [ ]:
joined = pl.from_arrow(read_table2).join(pl.from_arrow(gaf))

In [ ]:
# same well, channel
# start within (1s * 5000 kHz)

In [ ]:
read_table  # ["run_info"]

In [ ]:
read = next(reader.reads())

In [ ]:
read.run_info

In [ ]:
read.run_info.sample_rate

In [ ]:
read.pore

In [ ]:
read.start_sample

In [ ]:
with pod5.DatasetReader(pod5_filename) as reader:
    #for read in reader.reads():